In [26]:
import pandas as pd
import numpy as np

train = pd.read_csv("./pre_processed_train.csv",index_col="id")
test = pd.read_csv("./pre_processed_test.csv",index_col="id")

X=train.copy()
X_test=test.copy()


y=X.label
y_test=X_test.label

X=X.drop(['label'],axis=1)
X_test=X_test.drop(['label'],axis=1)

In [27]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from numpy import mean
from numpy import std

In [30]:
#LGBM and XGB best models
params_lgbm={'application':'binary','metric':'auc','num_leaves': 61, 'learning_rate': 0.09018623372462836, 'feature_fraction': 0.5345789610841638, 'bagging_fraction': 0.9913907896428181, 'max_depth': 5, 'max_bin': 58, 'min_sum_hessian_in_leaf': 39, 'subsample': 0.8}
model1 = LGBMClassifier(**params_lgbm)

params_xgb={'n_estimators': 3382, 'max_depth': 6, 'learning_rate': 0.009040061994740738, 'subsample': 0.9}
gpu_param={'eval_metric':'auc','n_jobs':-1, 'tree_method':'gpu_hist', 'predictor':"gpu_predictor",'gpu_id':0}
model2 = XGBClassifier(**params_xgb,**gpu_param)

params_gbm={'n_estimators': 3382, 'max_depth': 6, 'learning_rate': 0.009040061994740738, 'subsample': 0.9, 'min_samples_split': 320, 'n_iter_no_change':50,'validation_fraction':0.2}
model3 = GradientBoostingClassifier(**params_gbm)

In [32]:
# group / ensemble of models
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
warnings.warn('label_encoder_deprecation_msg', category=UserWarning)
estimator = []
estimator.append(('GBM', model3))
estimator.append(('LGBM', model1))
estimator.append(('XGB', model2))
  
vot_hard = VotingClassifier(estimators = estimator, voting ='hard',weights=[2,1,2])
vot_hard.fit(X, y)
y_pred = vot_hard.predict(X_test)

score = f1_score(y_test, y_pred,average='binary')
print("Voting Score %f" % score)


MemoryError: Unable to allocate 107. MiB for an array with shape (4952, 2838) and data type float64